# The Power of ZX-Calculus: A Practical Demonstration

This notebook demonstrates the capabilities of **PyZX**, a Python library for quantum reasoning using the ZX-calculus. Unlike standard circuit models based on matrices, ZX-calculus represents quantum processes as flexible graphs ("spiders" and wires), allowing for powerful topological simplifications.

## What You'll Learn

1. **Logic Cancellation**: Crushing a complex "hidden identity" circuit
2. **Visualizing Teleportation**: Proving teleportation is just a "bent wire"
3. **T-Count Reduction**: Optimizing fault-tolerant resources
4. **Stress Testing**: Reducing random Clifford+T circuits

## Prerequisites

You should have basic knowledge of:
- Quantum gates (Hadamard, CNOT, T-gate)
- Python programming
- (Optional) Basic linear algebra

---

## 1. Setup and Imports

We initialize the environment and set up the plotting backend.

In [ ]:
# Core imports
import pyzx as zx
import matplotlib.pyplot as plt

# Configure matplotlib for inline plotting
%matplotlib inline

# Set figure defaults
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print(f"PyZX Version: {zx.__version__}")
print("Setup complete! ✓")

---

## 2. The "Hidden Identity": Basic Simplification

In standard circuit notation, it's often difficult to see when a sequence of gates cancels out (e.g., Hadamards and CNOTs undoing each other). In ZX, this is trivial because the graph simply untangles.

We'll create a "messy" circuit that is mathematically equivalent to doing nothing (Identity), convert it to a graph, and simplify it.

### The Circuit

We'll build a 2-qubit circuit with a seemingly complex sequence of Hadamards and CNOTs that secretly does nothing.

In [ ]:
# Create a "messy" quantum circuit
# This sequence looks busy but evaluates to Identity
c_identity = zx.Circuit(qubit_amount=2)

c_identity.add_gate("HAD", 0)
c_identity.add_gate("CNOT", 0, 1)
c_identity.add_gate("HAD", 0)
c_identity.add_gate("HAD", 1)
c_identity.add_gate("CNOT", 0, 1)
c_identity.add_gate("HAD", 1)
c_identity.add_gate("CNOT", 0, 1)

print("=" * 50)
print("ORIGINAL CIRCUIT STATISTICS")
print("=" * 50)
print(c_identity.stats())

### Convert to ZX-Graph and Simplify

Now we convert this circuit to a ZX graph representation and apply the `full_reduce()` simplification strategy.

In [ ]:
# Convert to ZX-Graph
g_identity = c_identity.to_graph()

print(f"Graph before simplification: {g_identity.num_vertices()} vertices")

# Apply 'full_reduce' (The Master Simplification Strategy)
zx.full_reduce(g_identity)

# Verify Results
# A graph with 0 internal vertices is effectively an identity wire
is_identity = g_identity.num_vertices() == (g_identity.num_inputs() + g_identity.num_outputs())

print("\n" + "=" * 50)
print("AFTER ZX SIMPLIFICATION")
print("=" * 50)
print(f"Graph vertices: {g_identity.num_vertices()}")
print(f"Input vertices: {g_identity.num_inputs()}")
print(f"Output vertices: {g_identity.num_outputs()}")
print(f"\nIs strictly Identity (just wires)? {is_identity}")

if is_identity:
    print("\n✓ SUCCESS: The circuit collapsed to pure wires (Identity)!")
else:
    print("\n⚠ The circuit simplified but retained some structure.")

### Visualize the Result

Let's see what the simplified graph looks like. It should be straight lines connecting inputs to outputs.

In [ ]:
# Visualize the simplified graph
plt.figure(figsize=(8, 4))
zx.draw(g_identity)
plt.title("Simplified Graph (Should be Identity)")
plt.tight_layout()
plt.show()

### Key Takeaway

**What just happened?**

In standard circuit optimization, you would need to:
1. Pattern match specific gate sequences (like HZH = X)
2. Apply local rewrite rules repeatedly
3. Hope you find all the cancellations

In ZX-calculus:
1. Convert to graph (one step)
2. Apply topological simplification (automatic)
3. The algorithm "sees" the full structure and collapses it

---

## 3. Visualizing Teleportation: The "Bent Wire"

One of the most profound insights of Categorical Quantum Mechanics is that **Quantum Teleportation is topologically equivalent to a bent wire being straightened**.

We'll visualize the "Before" (Gates) and "After" (Topology) states, and then generate the optimized QASM.

### The Quantum Teleportation Protocol

Standard description:
1. Alice and Bob share an entangled Bell pair
2. Alice performs a Bell measurement on her qubit and the payload
3. Alice sends measurement results (classical bits) to Bob
4. Bob applies corrections based on Alice's measurements

ZX description:
- Information flows through a "bent wire"
- The Bell pair creates a "wormhole" in the graph
- Measurement + correction = straightening the wire

In [ ]:
# Construct the Teleportation Circuit
# Qubit 0: Payload (Alice's state to send)
# Qubit 1: Alice's half of entangled pair
# Qubit 2: Bob's half of entangled pair

c_teleport = zx.Circuit(qubit_amount=3)

# Step A: Create Entanglement between Alice (1) and Bob (2)
c_teleport.add_gate("HAD", 1)      # Hadamard on qubit 1
c_teleport.add_gate("CNOT", 1, 2)  # CNOT 1→2 (creates Bell pair)

# Step B: Alice performs Bell Measurement on Payload (0) and her Ancilla (1)
c_teleport.add_gate("CNOT", 0, 1)  # CNOT 0→1
c_teleport.add_gate("HAD", 0)      # Hadamard on qubit 0

# Note: In pure ZX logic (post-selected), we don't need classical corrections
# to see the information flow. The graph topology handles the channel.

print("Teleportation circuit created.")
print(f"Gates: {len(c_teleport.gates)}")

### Convert and Visualize: Before and After

We'll create a side-by-side comparison showing:
- **Left**: The raw protocol (gates as spiders)
- **Right**: The topological truth (simplified bent wire)

In [ ]:
# Convert to ZX-Graph
g_teleport = c_teleport.to_graph()

# Adjust graph positioning for clearer visualization
zx.simplify.to_clifford_normal_form_graph(g_teleport)

# Create side-by-side plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# PLOT 1: The Complex Reality (Before)
zx.draw(g_teleport, ax=ax1)
ax1.set_title("Before: The Raw Teleportation Protocol", fontsize=12, fontweight='bold')

# PLOT 2: The Topological Truth (After)
# Apply full_reduce to fuse the spiders
zx.full_reduce(g_teleport)

zx.draw(g_teleport, ax=ax2)
ax2.set_title("After: The 'Bent Wire' Straightened", fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Visualization complete!")
print("\nWhat you see:")
print("  - Left: Network of Green (Z) and Red (X) spiders")
print("  - Right: Simple wire from Input 0 to Output 2")
print("  - This proves: Teleportation = Identity channel")

### Generate QASM from Optimized Graph

Now let's convert the simplified graph back to QASM. Since the graph collapsed to essentially a wire (Identity), the QASM should be trivial.

In [ ]:
print("=" * 50)
print("CONVERTING GRAPH BACK TO CIRCUIT")
print("=" * 50)

# Extract the Circuit from the Graph
try:
    optimized_teleport = zx.extract_circuit(g_teleport.copy())
    print("✓ Extraction successful!\n")
except Exception as e:
    # Sometimes extraction requires the graph in a specific form
    print(f"Initial extraction failed: {e}")
    print("Converting to graph-like form...")
    zx.to_gh(g_teleport)
    optimized_teleport = zx.extract_circuit(g_teleport)
    print("✓ Extraction successful!\n")

# Generate QASM
qasm_string = optimized_teleport.to_qasm()

print("GENERATED QASM:")
print("-" * 50)
print(qasm_string)
print("-" * 50)

# Optional: Verify with Qiskit if available
try:
    from qiskit import QuantumCircuit
    
    qc = QuantumCircuit.from_qasm_str(qasm_string)
    print("\nQISKIT INTERPRETATION:")
    print(qc.draw(output='text'))
except ImportError:
    print("\n(Qiskit not installed - skipping visualization)")
except Exception as e:
    print(f"\n(Qiskit visualization failed: {e})")

### Key Takeaway

**What this proves:**

In standard quantum mechanics, teleportation is explained as:
- Complex entanglement operations
- Measurements collapsing superpositions
- Classical communication channels
- Conditional quantum gates

In ZX-calculus, we see the topological truth:
- **Teleportation is just routing information through a bent wire**
- The "cost" is zero (pure Identity operation)
- The entanglement creates a "shortcut" through the graph

This insight has led to new designs for quantum communication protocols and error correction schemes.

---

## 4. T-Count Reduction: The "Phase Gadget"

In **Fault-Tolerant Quantum Computing (FTQC)**, T-gates are expensive. They require "magic state distillation" which is ~1000× more costly than Clifford gates.

Standard compilers struggle to cancel T-gates separated by other structures. ZX-calculus treats T-gates as "spiders" that can slide through the web and fuse.

### The Example

We'll create a circuit with:
- A T-gate
- A CNOT (blocking structure)
- A T-dagger (inverse of T)

In standard logic, these gates can't "see" each other. In ZX, they fuse and annihilate.

In [ ]:
# Create a circuit with "expensive" T-gates
# T-gate = Z-rotation by π/4
# T-dagger = Z-rotation by -π/4

c_t = zx.Circuit(qubit_amount=2)

c_t.add_gate("T", 0)                # Expensive Gate 1
c_t.add_gate("CNOT", 0, 1)          # Blocking Structure
c_t.add_gate("T", 0, adjoint=True)  # Expensive Gate 2 (T-dagger)

print("=" * 50)
print("ORIGINAL CIRCUIT STATISTICS")
print("=" * 50)
print(f"T-count: {c_t.tcount()}")
print(f"Total gates: {len(c_t.gates)}")
print(c_t.stats())

### Apply ZX Optimization

Now we convert to ZX and apply `full_reduce()`. This will:
1. Convert T-gates into phase spiders (Green spiders with α = π/4)
2. Recognize that CNOT control is also a Green spider
3. Fuse T and T† through the CNOT
4. Calculate: π/4 + (-π/4) = 0 → Identity

In [ ]:
# Convert to ZX Graph
g_t = c_t.to_graph()

# The Optimization
zx.full_reduce(g_t)

# Extract and Verify
opt_c_t = zx.extract_circuit(g_t.copy())

print("\n" + "=" * 50)
print("OPTIMIZED CIRCUIT STATISTICS")
print("=" * 50)
print(f"T-count: {opt_c_t.tcount()}")
print(f"Total gates: {len(opt_c_t.gates)}")
print(opt_c_t.stats())

# Calculate reduction
if c_t.tcount() > 0:
    t_reduction = c_t.tcount() - opt_c_t.tcount()
    print(f"\n✓ T-gates removed: {t_reduction}")
    print(f"✓ T-count reduction: {100 * t_reduction / c_t.tcount():.1f}%")
    
    if opt_c_t.tcount() == 0:
        print("\n🎉 SUCCESS: All expensive T-gates eliminated!")

# Show resulting QASM
print("\n" + "=" * 50)
print("RESULTING QASM")
print("=" * 50)
print(opt_c_t.to_qasm())

### Key Takeaway

**Why this matters:**

In the simple example above, we saved 2 T-gates. But in real-world algorithms:

- **Shor's Algorithm**: ~10⁶ T-gates → 30% reduction saves 300,000 magic states
- **Chemistry Simulation**: ~10⁴ T-gates per iteration
- **Cryptographic Attacks**: ~10⁸ T-gates for meaningful security

Each T-gate requires:
- ~15 physical qubits for magic state distillation
- ~1000 physical operations
- Significant time overhead

**30% T-count reduction = 30% less hardware, 30% faster execution**

---

## 5. Stress Test: Random Clifford+T Circuit

Finally, let's stress-test ZX-calculus on a large, random circuit containing a mix of "cheap" Clifford gates and "expensive" T-gates.

This simulates a real-world compilation scenario where we have:
- A complex quantum algorithm
- Unoptimized output from an algorithm compiler
- Need to reduce resources before running on hardware

### Generate Random Chaos

We'll use PyZX's random circuit generator to create a Clifford+T circuit with:
- 5 qubits
- Depth 100 (many layers)
- 20% probability of T-gates (~20 expensive gates)

In [ ]:
print("=" * 60)
print("GENERATING RANDOM CLIFFORD+T CIRCUIT")
print("=" * 60)
print("Configuration:")
print("  - Qubits: 5")
print("  - Depth: 100")
print("  - T-gate probability: 20%")
print("\nGenerating...")

# Generate circuit
c_random = zx.generate.cliffordT(qubits=5, depth=100, p_t=0.2)

orig_gate_count = len(c_random.gates)
orig_t_count = c_random.tcount()

print("\n" + "=" * 60)
print("ORIGINAL CIRCUIT STATISTICS")
print("=" * 60)
print(f"Total gates: {orig_gate_count}")
print(f"T-count: {orig_t_count}")
print(f"Gate breakdown:")
print(c_random.stats())

### Optimize with ZX-Calculus

Now we apply the full power of ZX optimization. This may take a few seconds as the algorithm:
1. Converts the circuit to a graph
2. Applies spider fusion rules
3. Teleports phase gadgets through Cliffords
4. Cancels T-gates wherever possible

In [ ]:
print("Converting to ZX graph...")
g_random = c_random.to_graph()

print("Applying full_reduce optimization...")
print("(This may take a few seconds for large circuits)\n")

# The optimization
zx.full_reduce(g_random)

print("✓ Optimization complete!")
print("\nExtracting optimized circuit...")

# Extract back to circuit form
c_random_opt = zx.extract_circuit(g_random.copy())

new_gate_count = len(c_random_opt.gates)
new_t_count = c_random_opt.tcount()

print("✓ Extraction complete!")

### Analyze Results

Let's see how much optimization we achieved!

In [ ]:
print("\n" + "=" * 60)
print("OPTIMIZATION RESULTS")
print("=" * 60)

print("\nGate Count:")
print(f"  Original: {orig_gate_count}")
print(f"  Optimized: {new_gate_count}")
print(f"  Change: {new_gate_count - orig_gate_count:+d} gates")

if orig_gate_count > 0:
    gate_reduction = 100 * (orig_gate_count - new_gate_count) / orig_gate_count
    print(f"  Reduction: {gate_reduction:+.1f}%")

print("\nT-Count (The Critical Metric):")
print(f"  Original: {orig_t_count}")
print(f"  Optimized: {new_t_count}")
print(f"  Removed: {orig_t_count - new_t_count} T-gates")

if orig_t_count > 0:
    t_reduction = 100 * (orig_t_count - new_t_count) / orig_t_count
    print(f"  Reduction: {t_reduction:.1f}%")
    
    if t_reduction > 0:
        print(f"\n🎉 SUCCESS: Reduced T-count by {t_reduction:.1f}%!")
        print(f"\nIn FTQC, this saves:")
        saved_t = orig_t_count - new_t_count
        print(f"  - ~{saved_t * 15} physical qubits")
        print(f"  - ~{saved_t * 1000} physical operations")
else:
    print("\n(Original circuit had no T-gates)")

print("\n" + "=" * 60)
print("OPTIMIZED CIRCUIT BREAKDOWN")
print("=" * 60)
print(c_random_opt.stats())

### Verify Mathematical Correctness

It's crucial to verify that our optimization didn't break the circuit! We'll use PyZX's tensor comparison to prove both circuits implement the same quantum operation.

In [ ]:
print("=" * 60)
print("VERIFYING MATHEMATICAL EQUALITY")
print("=" * 60)
print("\nComparing tensor representations...")
print("(This compares the actual quantum operations)\n")

# This might take a moment as it computes tensor representations
try:
    # preserve_scalar=False allows for global phase differences (which don't matter physically)
    valid = zx.compare_tensors(c_random, c_random_opt, preserve_scalar=False)
    
    print("=" * 60)
    if valid:
        print("✓ VERIFICATION SUCCESSFUL")
        print("=" * 60)
        print("\nThe circuits are mathematically identical!")
        print("ZX optimization preserved correctness while reducing resources.")
    else:
        print("✗ VERIFICATION FAILED")
        print("=" * 60)
        print("\nWarning: Circuits are not mathematically identical.")
        print("This may indicate a bug or numerical precision issue.")
except Exception as e:
    print("⚠ VERIFICATION ERROR")
    print("=" * 60)
    print(f"\nCould not verify: {e}")
    print("(This is often due to circuit size - verification is computationally expensive)")

### Key Takeaway

**What we learned:**

Random circuits contain many "accidental" redundancies:

1. **Hidden Swaps**: Sequences like CNOT-CNOT-CNOT collapse to SWAP
2. **Phase Cancellations**: Random rotations often sum to 0° or 360°
3. **Commutation Opportunities**: Gates separated by layers can actually commute

ZX-calculus finds these automatically because:
- It treats the circuit as a **flexible graph**
- It ignores the artificial "layer" structure
- It looks for **topological equivalences** not local patterns

**Note on Gate Count vs T-Count:**

You may notice the total gate count sometimes *increases* while T-count decreases. This is intentional!

- In FTQC, CNOT gates are ~1000× cheaper than T-gates
- Trading 1 T-gate for 10 CNOTs is a massive win
- ZX optimization prioritizes T-count reduction above all else

---

## Conclusion

### What We Demonstrated

1. ✅ **Hidden Identity Detection**: ZX automatically detects when circuits cancel
2. ✅ **Topological Insights**: Teleportation is just a bent wire
3. ✅ **T-Count Reduction**: Critical optimization for fault-tolerant quantum computing
4. ✅ **Scalable Optimization**: Works on large, unstructured circuits

### The ZX Philosophy

**From Matrices to Graphs:**
- Traditional QC: Circuits are rigid sequences of matrix operations
- ZX-Calculus: Circuits are flexible graphs with topological freedom

**From Algebraic to Topological:**
- Traditional: Prove equivalence by matrix multiplication (exponential)
- ZX: Prove equivalence by graph rewriting (often polynomial)

**From Local to Global:**
- Traditional: Optimize by local pattern matching
- ZX: Optimize by seeing the entire structure at once

### Real-World Impact

ZX-calculus is not just academic theory. It's used in:

- **IBM Qiskit**: Transpiler optimization passes
- **Google Cirq**: Circuit synthesis algorithms
- **Quantinuum tket**: Native ZX-based compilation
- **Research**: Algorithm design, error correction, quantum ML

### Next Steps

To learn more:

1. **Read**: `INTRODUCTION-TO-ZX-CALCULUS.md` for theoretical foundations
2. **Experiment**: Modify the examples above with your own circuits
3. **Explore**: Try PyZX's other features (ZX-live, proof assistant)
4. **Apply**: Use ZX optimization in your quantum algorithms

### Resources

- **PyZX Documentation**: https://pyzx.readthedocs.io/
- **Original Papers**: Coecke & Duncan (2008, 2011)
- **Textbook**: "Picturing Quantum Processes" (Coecke & Kissinger)
- **Community**: ZX-calculus Discord, GitHub discussions

---

## Thank You!

**The future of quantum compilation is graphical.**

Happy optimizing! 🎉